In [1]:
import polars as pl
import pandas as pd

In [2]:
root_dir = '/home/nanni/unimore/tesi-magistrale/data/josie-tests/n45673-mset'

tables_csv_dir =        '/home/nanni/unimore/tesi-magistrale/data/turl_sloth_wikitables/sloth-subset/csv'
josie_sloth_ids_file =  root_dir + '/josie_sloth_ids.csv'
tables_file =           root_dir + '/tables.set'

In [3]:
josie_sloth_ids = pd.read_csv(josie_sloth_ids_file)

In [4]:
def _create_set_with_all_tokens(df, bag:bool=False):
    if not bag:
        if type(df) == pd.DataFrame:
            return {str(token).replace('\n', '\t').replace('|', ' ') for i in range(len(df.columns)) for token in df.iloc[:, i].unique() if not pd.isna(token) and token}
        elif type(df) == pl.DataFrame:
            pass

In [5]:
j_id1 = 15754
j_id2 = 26149

s_id1 = josie_sloth_ids[josie_sloth_ids['josie_id'] == j_id1]['sloth_id'].values[0]
s_id2 = josie_sloth_ids[josie_sloth_ids['josie_id'] == j_id2]['sloth_id'].values[0]

df1 = pd.read_csv(tables_csv_dir + '/' + s_id1, sep='\t')
df2 = pd.read_csv(tables_csv_dir + '/' + s_id2, sep='\t')

set1 = _create_set_with_all_tokens(df1)
set2 = _create_set_with_all_tokens(df2)

print(len(set1.intersection(set2)))

41


In [6]:
import re 

def extract_from_josie_results_pairs_and_overlaps(josie_results_file):
    josie_results = pl.scan_csv(josie_results_file).select(['query_id', 'results'])

    josie_results = josie_results \
        .with_columns(pl.col('results') \
                      .map_elements(lambda s: [int(x) for x in re.findall(r'\d+', s)], pl.List(pl.Int64))                      
                      ).with_columns(pl.col('results').list.to_struct()).unnest('results')                                                
    
    mapping = {
        'query_id': 'set_id',
        'field_0': 's1', 'field_1': 'o1', 
        'field_2': 's2', 'field_3': 'o2', 
        'field_4': 's3', 'field_5': 'o3'}
    
    return josie_results.collect().rename(mapping).sort(by=['o1', 'o2', 'o3'], descending=True).to_pandas()

In [8]:
res_top3 = extract_from_josie_results_pairs_and_overlaps('/home/nanni/unimore/tesi-magistrale/data/josie-tests/n45673-mset/result_k_3.csv')
res_top3.head()

,set_id,s1,o1,s2,o2,s3,o3
0,8681,77360,1391,9564,378,40437,336
1,76600,15779,1357,69763,1278,2833,1277
2,26179,69809,1344,71896,1280,47028,1120
3,69763,8038,1333,76600,1278,15779,1239
4,46562,23968,1315,2833,1302,71151,1233


In [ ]:
s1579o209s15754o208s57028o206

In [13]:
res_top3 = res_top3[['set_id', 's1', 'o1']]

In [15]:
l = []
for row in res_top3.itertuples(index=False):
    j_id1 = row[0]
    j_id2 = row[1]

    s_id1 = josie_sloth_ids[josie_sloth_ids['josie_id'] == j_id1]['sloth_id'].values[0]
    s_id2 = josie_sloth_ids[josie_sloth_ids['josie_id'] == j_id2]['sloth_id'].values[0]

    df1 = pd.read_csv(tables_csv_dir + '/' + s_id1, sep='\t')
    df2 = pd.read_csv(tables_csv_dir + '/' + s_id2, sep='\t')

    set1 = _create_set_with_all_tokens(df1)
    set2 = _create_set_with_all_tokens(df2)

    l.append(len(set1.intersection(set2)))

In [16]:
res_top3['myoverlap'] = l

In [17]:
res_top3

,set_id,s1,o1,myoverlap
0,8681,77360,1391,1473
1,76600,15779,1357,1344
2,26179,69809,1344,1337
3,69763,8038,1333,1298
4,46562,23968,1315,1303
...,...,...,...,...
89,10240,2586,27,27
90,38524,26304,27,27
91,64442,77301,27,27
92,55592,48342,25,7


In [19]:
s_id1.values[0]

'695403-2'